In [17]:
import json
from datasets import load_dataset
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from pinecone import Pinecone
from pinecone import ServerlessSpec
import time
from langchain.embeddings.openai import OpenAIEmbeddings
from tqdm.auto import tqdm
import langchain.vectorstores as vectorstore

In [18]:
# Create and Insert credentials in the file: SBDH-LLM/creds/azure_credentials.json
creds_file = "../credentials.json"
    
with open(creds_file, 'r') as file:
    creds_data = json.load(file)
    openai_api_key = creds_data['OPENAI_API_KEY']
    pinecone_api_key = creds_data['PINECONE_API_KEY']

assert openai_api_key != None, ""
assert pinecone_api_key != None, ""

In [19]:
chat  = ChatOpenAI(
    openai_api_key = openai_api_key,
    model = "gpt-3.5-turbo"
)

In [4]:
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I am great thank you, How can I help you?"),
    HumanMessage(content="I'd like to understand String Theory.")
]

In [5]:
res = chat(messages)

/data/mn27889/miniconda3/envs/rag-chatbot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [6]:
print(res.content)

String theory is a theoretical framework in physics that attempts to reconcile general relativity and quantum mechanics. It posits that the fundamental building blocks of the universe are not particles, but tiny, vibrating strings. These strings can have different vibrational modes, which correspond to different particles and forces in the universe.

One of the key ideas of string theory is that it requires more than the usual three spatial dimensions and one time dimension. In fact, string theory predicts the existence of extra dimensions beyond the familiar three dimensions of space and one dimension of time.

There are several different versions of string theory, such as Type I, Type IIA, Type IIB, heterotic SO(32), and heterotic E8xE8. These different versions are related through dualities, which suggest that they are actually different descriptions of the same underlying theory.

String theory is a complex and mathematically sophisticated theory that has the potential to provide a

In [7]:
messages.append(res)

In [8]:
messages

[SystemMessage(content='You are a helpful assistant'),
 HumanMessage(content='Hi AI, how are you today?'),
 AIMessage(content='I am great thank you, How can I help you?'),
 HumanMessage(content="I'd like to understand String Theory."),
 AIMessage(content='String theory is a theoretical framework in physics that attempts to reconcile general relativity and quantum mechanics. It posits that the fundamental building blocks of the universe are not particles, but tiny, vibrating strings. These strings can have different vibrational modes, which correspond to different particles and forces in the universe.\n\nOne of the key ideas of string theory is that it requires more than the usual three spatial dimensions and one time dimension. In fact, string theory predicts the existence of extra dimensions beyond the familiar three dimensions of space and one dimension of time.\n\nThere are several different versions of string theory, such as Type I, Type IIA, Type IIB, heterotic SO(32), and heterot

In [9]:
prompt = HumanMessage(
    content="Why do physicists believe it can prduce a unified theory?"
)

In [10]:
messages.append(prompt)

In [11]:
res = chat(messages)

In [12]:
print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it has the ability to describe all fundamental forces and particles in a single, coherent framework. Here are some reasons why physicists see string theory as a promising candidate for a unified theory:

1. **Incorporates Gravity**: Unlike quantum field theories, which struggle to incorporate gravity, string theory naturally includes gravity as one of the fundamental forces. This is important for achieving a unified theory that can describe all four fundamental forces (gravity, electromagnetism, weak nuclear force, and strong nuclear force) in a consistent manner.

2. **Resolves Infinities**: String theory has the potential to resolve the mathematical infinities that arise in quantum field theories, particularly in the context of gravity. By replacing point-like particles with extended objects (strings), string theory avoids certain infinities that plague traditional quantum field theories.

3. 

In [13]:
messages.append(res)

In [14]:
messages

[SystemMessage(content='You are a helpful assistant'),
 HumanMessage(content='Hi AI, how are you today?'),
 AIMessage(content='I am great thank you, How can I help you?'),
 HumanMessage(content="I'd like to understand String Theory."),
 AIMessage(content='String theory is a theoretical framework in physics that attempts to reconcile general relativity and quantum mechanics. It posits that the fundamental building blocks of the universe are not particles, but tiny, vibrating strings. These strings can have different vibrational modes, which correspond to different particles and forces in the universe.\n\nOne of the key ideas of string theory is that it requires more than the usual three spatial dimensions and one time dimension. In fact, string theory predicts the existence of extra dimensions beyond the familiar three dimensions of space and one dimension of time.\n\nThere are several different versions of string theory, such as Type I, Type IIA, Type IIB, heterotic SO(32), and heterot

In [15]:
prompt = HumanMessage(
    content="Why do you know about LLama2?"
)

messages.append(prompt)

res = chat(messages)

In [16]:
print(res.content)

I'm sorry, but I don't have information about LLama2. It seems like you may be referring to something specific or asking about a topic that I am not familiar with. If you can provide more context or details, I'd be happy to try to help or provide information on a related topic.


In [17]:
messages.append(res)

In [18]:
prompt = HumanMessage(
    content="Can you tell me more about the LLMChain in LangChain?"
)

messages.append(prompt)

res = chat(messages)

In [19]:
print(res.content)

I'm sorry, but I am not familiar with the specific terms "LLMChain" or "LangChain." It's possible that they are related to a specific project, technology, or concept that I may not have information about. If you can provide more context or details, I'd be happy to try to help or provide information on a related topic.


In [20]:
llmchain_information = [
    "A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.",
    "Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.",
    "LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is designed with the objective in mind to enable those types of applications."
]

source_knowledge = "\n".join(llmchain_information)

In [21]:
query = "Can you tell me about the LLMChain in LangChain?"

augmented_query = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

In [22]:
prompt = HumanMessage(
    content=augmented_query
)

messages.append(prompt)

res = chat(messages)

In [23]:
print(res.content)

The LLMChain in LangChain is a common type of chain within the LangChain framework for developing applications powered by language models. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. The LLMChain takes multiple input variables, utilizes the PromptTemplate to format them into a prompt, passes that prompt to the model for processing, and then uses the OutputParser (if provided) to parse the output of the LLM into a final format.

In the context of LangChain, a chain refers to a sequence of modular components or other chains that are combined in a specific way to achieve a common use case. LangChain aims to enable the development of powerful and differentiated applications that go beyond just calling out to a language model via an API. These applications are designed to be data-aware, connecting language models to other sources of data, and agentic, allowing language models to interact with their environment.

Overall, the LLMCha

### Building the knowledgebase in PineCone

In [24]:
dataset = load_dataset(
    "jamescalam/llama-2-arxiv-papers-chunked",
    split = "train"
)

In [25]:
dataset[0]

{'doi': '1102.0183',
 'chunk-id': '0',
 'chunk': 'High-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nTechnical Report No. IDSIA-01-11\nJanuary 2011\nIDSIA / USI-SUPSI\nDalle Molle Institute for Arti\x0ccial Intelligence\nGalleria 2, 6928 Manno, Switzerland\nIDSIA is a joint institute of both University of Lugano (USI) and University of Applied Sciences of Southern Switzerland (SUPSI),\nand was founded in 1988 by the Dalle Molle Foundation which promoted quality of life.\nThis work was partially supported by the Swiss Commission for Technology and Innovation (CTI), Project n. 9688.1 IFF:\nIntelligent Fill in Form.arXiv:1102.0183v1  [cs.AI]  1 Feb 2011\nTechnical Report No. IDSIA-01-11 1\nHigh-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nJanuary 2011\nAbs

In [5]:
pc_client = Pinecone(api_key=pinecone_api_key)

In [6]:
index_name = "openai-rag-chatbot"

if index_name not in pc_client.list_indexes().names():
    pc_client.create_index(
        name=index_name,
        dimension=1536, # dimensionality of ada 002
        metric="cosine", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    )
    
while not pc_client.describe_index(index_name).status['ready']:
    time.sleep(1)

In [7]:
# Connect to index
index = pc_client.Index(index_name)
time.sleep(1)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

In [8]:
embed_model = OpenAIEmbeddings(api_key=openai_api_key, model="text-embedding-ada-002")

/data/mn27889/miniconda3/envs/rag-chatbot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
texts = [
    "this is the first chunk of the text",
    "then anothe second chunk of the text is here"
]

In [10]:
res = embed_model.embed_documents(texts)

In [11]:
len(res), len(res[0])

(2, 1536)

In [52]:
# data = dataset.to_pandas()

# batch_size = 100

# for i in tqdm(range(0, len(data), batch_size)):
#     i_end = min(len(data), i+batch_size)
    
#     batch = data.iloc[i:i_end]
    
#     ids = [f"{x['doi']}-{x['chunk-id']}" for i,x in batch.iterrows()]
    
#     texts = [x['chunk'] for _, x in batch.iterrows()]
    
#     embeds = embed_model.embed_documents(texts)
    
#     metadata = [
#         {
#             'text': x['chunk'],
#             'source': x['source'],
#             'title': x['title']
#         } for i,x in batch.iterrows()
#     ]
    
#     index.upsert(vectors=zip(ids, embeds, metadata))

100%|██████████| 49/49 [01:10<00:00,  1.45s/it]


In [12]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

### Retrieval Augmented Generation

In [21]:
text_field = "text" # the metadata field that contains our text
vectorstore = vectorstore.Pinecone(index, embed_model.embed_query, text_field)

/data/mn27889/miniconda3/envs/rag-chatbot/lib/python3.11/site-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [14]:
query = "What is so special about Llama 2"
vectorstore.similarity_search(query, k=3)

[Document(metadata={'source': 'http://arxiv.org/pdf/2307.09288', 'title': 'Llama 2: Open Foundation and Fine-Tuned Chat Models'}, page_content='Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic\nSergey Edunov Thomas Scialom\x03\nGenAI, Meta\nAbstract\nIn this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned\nlarge language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.\nOur ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our\nmodels outperform open-source chat models on most benchmarks we tested, and based on\nourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tu

In [22]:
def augment_prompt(query: str):
    # Get Top 3 results
    results = vectorstore.similarity_search(query, k=3)
    
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    
    augmented_prompt = f"""Using the contexts below, answer the query.
    
    Contexts:
    {source_knowledge}
    
    Query: {query}"""
    
    return augmented_prompt

In [25]:
final_query = augment_prompt(query)

In [26]:
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I am great thank you, How can I help you?"),
    HumanMessage(content=final_query)
]

In [27]:
res = chat(messages)

/data/mn27889/miniconda3/envs/rag-chatbot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [28]:
print(res.content)

Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters. The fine-tuned LLMs, specifically L/l.sc/a.sc/m.sc/a.sc/two.taboldstyle-C/h.sc/a.sc/t.sc, are optimized for dialogue use cases. In benchmarks tested, these models outperform open-source chat models and even demonstrate potential as substitutes for closed-source models based on human evaluations for helpfulness and safety. The fine-tuning and safety approaches taken with Llama 2 are detailed and aim to enhance usability and safety without the significant costs and lack of transparency often associated with closed-source models.
